In [1]:
import pandas as pd

cleaner_data = pd.read_csv('cleaner_data.csv')
cleaner_data

,Label,Sentences
0,0,think hate summer
1,1,time subway run
2,0,feel terrible definitely sick
3,0,not wait dare quotrealquot job probably go wai...
4,0,battery low20iphone 3 g realy battery killer
...,...,...
1491034,1,enjoy new old dub funk acquire
1491035,1,weather awesome
1491036,1,margaret
1491037,0,chris sweet weepy aww wish people not hard tim...


In [8]:
# Splitting data according to a 70:15:15 ratio

remaining_data = cleaner_data[:int(len(cleaner_data)*0.95)]
test_data = cleaner_data[int(len(cleaner_data)*0.95):]

useful_data = remaining_data[:400000]
training_data = useful_data[:int(len(useful_data)*0.8)]
validation_data = useful_data[int(len(useful_data)*0.8):]
                                     
training_data

,Label,Sentences
0,0,think hate summer
1,1,time subway run
2,0,feel terrible definitely sick
3,0,not wait dare quotrealquot job probably go wai...
4,0,battery low20iphone 3 g realy battery killer
...,...,...
319995,1,beautiful weather
319996,0,short
319997,1,ok go bed end tweet convo ttyl hun
319998,1,happy glad find find fantastic go


In [9]:
import numpy as np
    
train_x_val = np.asarray(training_data["Sentences"])
train_y_val = np.asarray(training_data["Label"])

valid_x_val = np.asarray(validation_data["Sentences"])
valid_y_val = np.asarray(validation_data["Label"])

In [10]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

#embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
# embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1"
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"

hub_layer = hub.KerasLayer(embedding, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)
# hub_layer(training_data["Sentences"][:3])

In [13]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_11 (Dense)             (None, 100)               2100      
_________________________________________________________________
dense_12 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_13 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_14 (Dense)             (None, 10)                260       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 408,716
Trainable params: 408,716
Non-trainable params: 0
________________________________________________

In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_x_val,
                    train_y_val,
                    epochs=20,
                    validation_data=(valid_x_val, valid_y_val),
                    verbose=1)